In [2]:
# load data from https://huggingface.co/datasets/BytedTsinghua-SIA/DAPO-Math-17k

import datasets
dataset = datasets.load_dataset("BytedTsinghua-SIA/DAPO-Math-17k", split="train")

/lustrefs/users/shibo.hao/miniforge3/envs/Reasoning360/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 1791700/1791700 [00:19<00:00, 89867.85 examples/s] 


In [5]:
import pandas as pd
df = pd.DataFrame(dataset)
# Deduplicate based on the index in extra_info
df['index'] = df['extra_info'].apply(lambda x: x['index'])
df_deduped = df.drop_duplicates(subset=['index'])

# Convert back to HuggingFace dataset
deduped_dataset = datasets.Dataset.from_pandas(df_deduped)

print(f"Original dataset size: {len(dataset)}")
print(f"Deduplicated dataset size: {len(deduped_dataset)}")

Original dataset size: 1791700
Deduplicated dataset size: 17917


In [7]:
# save the deduplicated dataset
deduped_dataset.save_to_disk("deduplicated_dapo_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 17917/17917 [00:00<00:00, 546620.20 examples/s]


In [16]:
# add a new column "original_question"
df_deduped['original_question'] = df_deduped['prompt'].apply(
    lambda x: x[0]['content'].split('Answer is the answer to the problem.\n\n')[1].split('\n\nRemember to put your answer on its own line after')[0].strip()
)

df_deduped['prompt'] = df_deduped['original_question'].apply(
    lambda x: [
        {
            "role": "user",
            "content": x + " Please output the final answer within \\boxed{}."
        }
    ]
)

/tmp/ipykernel_3150576/2521534035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduped['original_question'] = df_deduped['prompt'].apply(
/tmp/ipykernel_3150576/2521534035.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduped['prompt'] = df_deduped['original_question'].apply(


In [17]:
deduped_dataset = datasets.Dataset.from_pandas(df_deduped)

In [19]:
# deduped_dataset = datasets.Dataset.from_pandas(df_deduped)
deduped_dataset.save_to_disk("deduplicated_dapo_dataset")
# upload to my huggingface account
deduped_dataset.push_to_hub("SDSB/deduplicated_dapo_dataset")


Saving the dataset (1/1 shards): 100%|██████████| 17917/17917 [00:00<00:00, 268451.88 examples/s]


Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.99s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/SDSB/deduplicated_dapo_dataset/commit/bc74ffe1a450053df18db8656add2dbba0a9f90f', commit_message='Upload dataset', commit_description='', oid='bc74ffe1a450053df18db8656add2dbba0a9f90f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SDSB/deduplicated_dapo_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SDSB/deduplicated_dapo_dataset'), pr_revision=None, pr_num=None)

In [47]:
# load Skywork/Skywork-OR1-RL-Data

dataset = datasets.load_dataset("Skywork/Skywork-OR1-RL-Data", split="math")

print(dataset[0])

{'data_source': 'train-math-numinamath1.5_olympiads', 'prompt': [{'content': '33. How many six-digit numbers are there in which all digits are odd?', 'role': 'user'}], 'ability': 'math', 'reward_model': {'ground_truth': '["15625"]', 'style': 'rule'}, 'extra_info': {'index': 7, 'model_difficulty': {'DeepSeek-R1-Distill-Qwen-1.5B': 2, 'DeepSeek-R1-Distill-Qwen-32B': 2, 'DeepSeek-R1-Distill-Qwen-7B': 1}}}


In [48]:
# add original_question column
dataset_df = pd.DataFrame(dataset)

dataset_df['original_question'] = dataset_df['prompt'].apply(
    lambda x: x[0]['content'].strip()
)

dataset_df['prompt'] = dataset_df['original_question'].apply(
    lambda x: [
        {
            "role": "user",
            "content": x + " Please output the final answer within \\boxed{}."
        }
    ]
)

dataset = datasets.Dataset.from_pandas(dataset_df)
dataset.save_to_disk("or1_dataset")
dataset.push_to_hub("SDSB/or1_dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:08<00:00,  8.56s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/SDSB/or1_dataset/commit/34ba67440e35c075cd04f2ee1b6e42de38546003', commit_message='Upload dataset', commit_description='', oid='34ba67440e35c075cd04f2ee1b6e42de38546003', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SDSB/or1_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SDSB/or1_dataset'), pr_revision=None, pr_num=None)

In [50]:
dataset[0]

{'data_source': 'train-math-numinamath1.5_olympiads',
 'prompt': [{'content': '33. How many six-digit numbers are there in which all digits are odd? Please output the final answer within \\boxed{}.',
   'role': 'user'}],
 'ability': 'math',
 'reward_model': {'ground_truth': '["15625"]', 'style': 'rule'},
 'extra_info': {'index': 7,
  'model_difficulty': {'DeepSeek-R1-Distill-Qwen-1.5B': 2,
   'DeepSeek-R1-Distill-Qwen-32B': 2,
   'DeepSeek-R1-Distill-Qwen-7B': 1}},
 'original_question': '33. How many six-digit numbers are there in which all digits are odd?'}

In [51]:
dataset_or1 = datasets.load_dataset("SDSB/or1_dataset")
dataset_dapo = datasets.load_dataset("SDSB/deduplicated_dapo_dataset")


Generating train split: 100%|██████████| 105055/105055 [00:00<00:00, 460320.52 examples/s]


In [52]:
dataset_dapo

DatasetDict({
    train: Dataset({
        features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'index', 'original_question', '__index_level_0__'],
        num_rows: 17917
    })
})

In [53]:
# print keys
print(dataset_dapo['train'])
print(dataset_or1['train'])

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'index', 'original_question', '__index_level_0__'],
    num_rows: 17917
})
Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'original_question'],
    num_rows: 105055
})


In [54]:
dataset_or1['train'][0]['original_question']

'33. How many six-digit numbers are there in which all digits are odd?'

In [57]:
from tqdm import tqdm
new_dataset = []
raw_question_set = set()
for question in tqdm(dataset_or1['train']['original_question']):
    # print(question)
    if len([question in question_record for question_record in raw_question_set]) == 0 and len([question_record in question for question_record in raw_question_set]) == 0:
        new_dataset.append(question)
    
    raw_question_set.add(question)
    # break

100%|██████████| 105055/105055 [18:54<00:00, 92.57it/s] 


In [ ]:
new_dataset

In [58]:
len(new_dataset)

1

In [43]:
len(new_dataset)

1

In [36]:
dataset_or1['train']

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'original_question'],
    num_rows: 105055
})

In [37]:
dataset_merged = datasets.concatenate_datasets([dataset_or1['train'], dataset_dapo['train']])


ValueError: The features can't be aligned because the key index of features {'index': Value(dtype='string', id=None)} has unexpected type - Value(dtype='string', id=None) (expected either Value(dtype='int64', id=None) or Value("null").